####  Please note that this notebook was used in early phases of the project and it is just added to the GitHub repo for reference, this is why it does not reflect the final model data preparation code used.

In [ ]:
!!pip install -q git+https://github.com/keras-team/keras-nlp.git --upgrade

In [ ]:
import keras_nlp

Using TensorFlow backend


In [ ]:
!!pip install keras-tuner -q

['\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/128.9 kB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━━━━\x1b\x1b╸\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b30.7/128.9 kB\x1b \x1b1.1 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b128.9/128.9 kB\x1b \x1b2.6 MB/s\x1b eta \x1b0:00:00\x1b',
 '\x1b[?25h']

In [ ]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split

# Load and Prepare Data

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/PSUT/Graduation-Project2/rehab-ai-data/KiMoRe_final/KiMoRe_Es4_movenet.csv")

In [ ]:
df

,ID,exercise,video,joint_positions,clinical_score,#frames
0,P_ID11,Es4,/content/drive/MyDrive/PSUT/Graduation-Project...,/content/drive/MyDrive/PSUT/Graduation-Project...,15.333333,363
1,P_ID16,Es4,/content/drive/MyDrive/PSUT/Graduation-Project...,/content/drive/MyDrive/PSUT/Graduation-Project...,30.000000,746
2,P_ID10,Es4,/content/drive/MyDrive/PSUT/Graduation-Project...,/content/drive/MyDrive/PSUT/Graduation-Project...,32.149872,563
3,P_ID4,Es4,/content/drive/MyDrive/PSUT/Graduation-Project...,/content/drive/MyDrive/PSUT/Graduation-Project...,14.000000,941
4,P_ID3,Es4,NaN,NaN,-1.000000,0
...,...,...,...,...,...,...
73,E_ID4,Es4,/content/drive/MyDrive/PSUT/Graduation-Project...,/content/drive/MyDrive/PSUT/Graduation-Project...,44.333333,538
74,E_ID6,Es4,/content/drive/MyDrive/PSUT/Graduation-Project...,/content/drive/MyDrive/PSUT/Graduation-Project...,42.666667,415
75,E_ID1,Es4,/content/drive/MyDrive/PSUT/Graduation-Project...,/content/drive/MyDrive/PSUT/Graduation-Project...,50.000000,1422
76,E_ID7,Es4,/content/drive/MyDrive/PSUT/Graduation-Project...,/content/drive/MyDrive/PSUT/Graduation-Project...,44.333333,554


In [ ]:
def get_dataframe_cols():
  KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
  }
  df_cols = []
  for keypoint_name in KEYPOINT_DICT:
    df_cols.append(f"{keypoint_name}_y")
    df_cols.append(f"{keypoint_name}_x")
    df_cols.append(f"{keypoint_name}_confidence")
  return df_cols

In [ ]:
all_cols = get_dataframe_cols()
face_cols = all_cols[:15]
cols_drop = face_cols
print(f"Dropping {len(cols_drop)} columns.")

Dropping 15 columns.


In [ ]:
EXERCISE = "Es4"
EXERCISE_VIDO_MAX_LEN = df[df['exercise'] == EXERCISE]['#frames'].max()
print(f"Maximum video length: {EXERCISE_VIDO_MAX_LEN}")

def prepare_data(df, exercise_video_max_len, data_type):
  data = []
  labels = []
  padding_masks = []

  for index, row in df.iterrows():
    joint_positions_path = row['joint_positions']
    if joint_positions_path is np.NAN:
      continue
    clinical_score = row['clinical_score']
    video_length = row['#frames']

    joint_positions_data = pd.read_csv(joint_positions_path)
    joint_positions_data = joint_positions_data.drop(cols_drop, axis=1)
    joint_positions_data = joint_positions_data.to_numpy()

    padding_length = exercise_video_max_len - video_length
    padding_mask = np.zeros((video_length + padding_length))
    padding_mask[-padding_length:] = 1

    joint_positions_data_padded = np.pad(joint_positions_data, ((0, padding_length), (0, 0)), mode='constant', constant_values=0)

    data.append(joint_positions_data_padded)
    labels.append(clinical_score)
    padding_masks.append(padding_mask)

  data = np.array(data)
  labels = np.array(labels)
  padding_masks = np.array(padding_masks)

  data = np.nan_to_num(data)
  labels = np.nan_to_num(labels)

  print(f"{data_type} Data Shape:", data.shape)
  print(f"{data_type} Labels Shape:", labels.shape)
  print(f"{data_type} Padding Masks Shape:", padding_masks.shape)

  return (data, padding_masks), labels

Maximum video length: 1988


In [ ]:
train, test = train_test_split(df, test_size=0.2, random_state=0)
train_data, train_labels = prepare_data(train, EXERCISE_VIDO_MAX_LEN, "Train")
test_data, test_labels = prepare_data(test, EXERCISE_VIDO_MAX_LEN, "Test")

Train Data Shape: (57, 1988, 36)
Train Labels Shape: (57,)
Train Padding Masks Shape: (57, 1988)
Test Data Shape: (15, 1988, 36)
Test Labels Shape: (15,)
Test Padding Masks Shape: (15, 1988)


# Build Model

In [ ]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner

In [ ]:
def build_model(hp):
  num_windows=4
  window_size=497
  num_heads=hp.Int("num_heads", 1, 5)
  d_model=hp.Int("d_model", 5, 20)

  inputs = tf.keras.Input(shape=(1988, 36), name='orignal_data')
  masks = tf.keras.Input(shape=(1988), name='padding_masks')

  windows = tf.split(inputs, num_windows, axis=1)
  windows_masks = tf.split(masks, num_windows, axis=1)

  embedding_layers = []
  for i in range(hp.Int("num_embedding_layers", 1, 4)):
    embedding_layer = tf.keras.layers.Dense(
                                units=hp.Int("units", min_value=5, max_value=20, step=1),
                                activation="relu",
                            )
    embedding_layers.append(embedding_layer)

  final_embedding_layer = tf.keras.layers.Dense(units=d_model, activation="relu")
  embedding_layers.append(final_embedding_layer)

  embeddings = []
  for window in windows:
    for i, embedding_layer in enumerate(embedding_layers):
      embedding = embedding_layer(window)
    if hp.Boolean(f"dropout_embedding{i}"):
      embedding_layer = tf.keras.layers.Dropout(
                        rate=hp.Float(f"dropout_embedding{i}_rate",
                        min_value=0.20,
                        max_value=0.50,
                        sampling="log"))(embedding)
    embeddings.append(embedding)

  positional_embedding_layer = tf.keras.layers.Embedding(input_dim=window_size, output_dim=d_model)
  positional_embeddings = []
  for i in range(num_windows):
      positional_embedding = positional_embedding_layer(tf.range(window_size))
      positional_embeddings.append(positional_embedding)

  embeddings_all = [embedding + positional_embedding for embedding, positional_embedding in zip(embeddings, positional_embeddings)]

  transformer_encoder_layer = keras_nlp.layers.TransformerEncoder(intermediate_dim=d_model, num_heads=num_heads)
  encoded = [transformer_encoder_layer(window_embd, window_mask) for window_embd, window_mask in zip(embeddings, windows_masks)]

  concat_output = tf.concat(encoded, axis=1)
  flatten_output = tf.keras.layers.Flatten()(concat_output)

  if hp.Boolean("dropout_flatten"):
    flatten_output = tf.keras.layers.Dropout(rate=hp.Float("dropout_flatten_rate", min_value=0.20, max_value=0.50, sampling="log"))(flatten_output)

  dense_output = tf.keras.layers.Dense(
                              units=hp.Int("units", min_value=8, max_value=9940, step=8),
                              activation="relu",
                          )(flatten_output)

  for i in range(hp.Int("num_dense_layers1", 1, 5)):
    dense_output = tf.keras.layers.Dense(
                                units=hp.Int("units", min_value=32, max_value=512, step=32),
                                activation="relu",
                            )(dense_output)
    if hp.Boolean(f"dropout_dense_layers1_{i}"):
      dense_output = tf.keras.layers.Dropout(rate=hp.Float(f"dropout_dense_layers1_{i}_rate", min_value=0.20, max_value=0.50, sampling="log"))(dense_output)

  for i in range(hp.Int("num_dense_layers2", 1, 5)):
    dense_output = tf.keras.layers.Dense(
                                units=hp.Int("units", min_value=32, max_value=512, step=32),
                                activation="relu",
                            )(dense_output)
    if hp.Boolean(f"dropout_dense_layers2_{i+1}"):
      dense_output = tf.keras.layers.Dropout(rate=hp.Float(f"dropout_dense_layers2_{i+1}_rate", min_value=0.20, max_value=0.50, sampling="log"))(dense_output)


  output = tf.keras.layers.Dense(1)(dense_output)

  model = tf.keras.Model(inputs=[inputs, masks],
                       outputs=output,
                       name='transformer_model')

  learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
      loss='mse',
      metrics=['mae']
      )

  return model

In [ ]:
tuner = keras_tuner.BayesianOptimization(
    hypermodel=build_model,
    objective=["mae","val_mae"],
    max_trials=40,
    overwrite=True,
    directory="/content/drive/MyDrive/PSUT/Graduation-Project2/code/models/hyperparameter-tuning",
    project_name="/content/drive/MyDrive/PSUT/Graduation-Project2/code/models/hyperparameter-tuning/Experiment2",
)

In [ ]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective=["mae","val_mae"],
    max_trials=50,
    executions_per_trial=1,
    overwrite=True,
    directory="/content/drive/MyDrive/PSUT/Graduation-Project2/code/models/hyperparameter-tuning",
    project_name="/content/drive/MyDrive/PSUT/Graduation-Project2/code/models/hyperparameter-tuning/Experiment3"
)

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 11
num_heads (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 5, 'step': 1, 'sampling': 'linear'}
d_model (Int)
{'default': None, 'conditions': [], 'min_value': 5, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
num_embedding_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 4, 'step': 1, 'sampling': 'linear'}
units (Int)
{'default': None, 'conditions': [], 'min_value': 5, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
dropout_embedding1 (Boolean)
{'default': False, 'conditions': []}
dropout_flatten (Boolean)
{'default': False, 'conditions': []}
num_dense_layers1 (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 5, 'step': 1, 'sampling': 'linear'}
dropout_dense_layers1_0 (Boolean)
{'default': False, 'conditions': []}
num_dense_layers2 (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 5, 'step': 1, 'sampling': 'linear'}
dropout_dense_layers2_1 

In [ ]:
tuner.search([train_data[0], train_data[1]], train_labels, epochs=20, validation_data=([test_data[0], test_data[1]], test_labels))

Trial 50 Complete [00h 02m 32s]
multi_objective: 41.89490509033203

Best multi_objective So Far: 16.85331153869629
Total elapsed time: 01h 50m 38s


In [ ]:
tuner.results_summary()

Results summary
Results in /content/drive/MyDrive/PSUT/Graduation-Project2/code/models/hyperparameter-tuning
Showing 10 best trials
Objective(name="val_mae", direction="min")

Trial 0 summary
Hyperparameters:
num_heads: 1
d_model: 8
num_embedding_layers: 4
units: 15
dropout_embedding1: True
dropout_flatten: True
num_dense_layers1: 2
dropout_dense_layers1_0: False
num_dense_layers2: 1
dropout_dense_layers2_1: False
lr: 0.0037849837509274843
dropout_embedding4: False
dropout_flatten_rate: 0.2
dropout_dense_layers1_1: False
Score: 6.889535427093506

Trial 1 summary
Hyperparameters:
num_heads: 1
d_model: 20
num_embedding_layers: 3
units: 11
dropout_embedding1: False
dropout_flatten: False
num_dense_layers1: 2
dropout_dense_layers1_0: True
num_dense_layers2: 2
dropout_dense_layers2_1: False
lr: 0.0001157452518760688
dropout_embedding4: True
dropout_flatten_rate: 0.24555001107823002
dropout_dense_layers1_1: False
dropout_embedding3: False
dropout_dense_layers1_0_rate: 0.2
dropout_dense_layer